**Наверное, каждый DS специалист знает как работают bagging & ensemble algorithms in ML. Все хором скажут, что это алгоритмы, которые обучают n-ое кол-во моделей(weak models / estimators) и делают финальное предсказание на основе усреднения,голосования полученных значений. Да,правильно. Однако, интересен тот факт, что модели градиентного бустинга учатся не на истинных значениях('y' labels),а на "остатках" от предсказания модели с предыдущего шага. Если же говорить про регрссию,то таргетом модели всегда является ошибка предыдущей модели, под которую старается подстроиться следующая модель. Звучит не очень понятно и костыльно, но в jupyter notebook это выглядит весьма тривиально. Сейчас постараюсь это вам показать.**

> Также как и в предыдущей серии проекта необходимо смотреть не только на .ipynb файл, но и на .py . В текущей серии все внимание должно быть приковано именно к ним, так как там реализована вся техника градиентного бустинга, который я попытался кратко описать выше.

### Немного теории перед практикой

* **Алгоритм градиентного бустинга с выводом антиградиента с косинусной функцией потерь:**
    * Мы решаем задачу, где $yi$ - целевая переменная, а $f(xi)$ - предсказание модели.
    * Косинусная функция потерь:
        * $L(yi,f(xi)) = cos(yi, f(xi))$  $<=>$  $\frac{\partial L(y_i, f(x_i))}{\partial f(x_i)} = -\sin(y_i - f(x_i))$  $<=>$  $r_{im} = -\sin(y_i - f(x_i))$
        
    1) **Инициализация алгоритма**:
       * Инициализировать начальную модель: $f$<sub>0</sub>$(x)$ константным значением.
    2) **Пошаговое обучение модели**:
       * Для каждого шага $m=1,2,3,...,M$ , где  $M$ - количество деревьев:
           * Вычислить антиградиент косинусной функции потерь по предсказаниям: $r_{im} = -\sin(y_i - f(x_i))$
           * Обучить дерево решений $h$<sub>m</sub>$(x)$ на антиградиенте $r$<sub>im</sub>
           * Вычислить оптимальный коэффициент для дерева $m$:
               * $\gamma_m = \arg \min_\gamma \sum_{i=1}^{N} L(y_i, f_{m-1}(x_i) + \gamma h_m(x_i))$
                   - $N$ - количество наблюдений в обучающем наборе
                   - $f_{m-1}(x_i)$ - текущее предсказание модели (комбинация предыдущих деревьев)
                   - $h_m(x_i)$ - предсказание текущего дерева
                   - $L(y_i, f_{m-1}(x_i) + \gamma h_m(x_i))$ - косинусная функция потерь между истинным значением $y_i$ и комбинированным предсказанием.
           * Обновить модель, двигаясь в сторону антиградиента:
               * $f_m(x) = f_{m-1}(x) + \eta \gamma_m h_m(x)$
                   - $f$<sub>m</sub>$(x)$ - новое предсказание на $m$-м шаге
                   - $f$<sub>m-1</sub>$(x)$ - предыдущее предсказание
                   - $η$ - темп обучения (learning rate)
                   - $γ$<sub>m</sub> - оптимальный коэффициент для $m$-го дерева
                   - $h$<sub>m</sub>$(x)$ - предсказание $m$-го дерева
                   
    3) **Вывод оптимизирующей функции**:
        * $F(x) = \sum_{m=1}^{M} \gamma_m h_m(x)$
            - Главная задача данного метода - минимизация косинусной функции потерь при последовательном добавлении деревьев


# Начинаются блоки кодинга

In [ ]:
from DecisionTreeRegressor import DecisionTreeRegressor
from GradientBoosting import GradientBoosting
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes

In [ ]:
# Берем готовые датасет из sklearn 
data = load_diabetes()

In [ ]:
X,y = data.data, data.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.3, random_state=42)

> Для начала посмотрим на performance, который мы получим применив Decision Tree. Однако, стоить предупредить, что дерево решений для данной задачи было переписано именно под регрессию.

### Decision Tree Regressor

In [ ]:
# Создаем экземпляр класса DecisionTreeRegressor
tree = DecisionTreeRegressor(criterion = 'mse', splitter = 'best', min_samples_leaf = 5)

# Обучаем дерево
tree.fit(X_train, y_train)

# Делаем предсказание
y_pred = tree.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"Mape: {mape}")

# Проверка дерева на датасете перед построением ансамбля

### Gradient Boosting

In [ ]:
# args для создания безлайна бустинга
n_estimators = 100
learning_rate = 0.1
splitter = 'random' # Оставлю random, чтобы сделать безлайн
min_samples_leaf = 1
criterion = 'mse'
max_features = None


gb_model = GradientBoosting(n_estimators = n_estimators, learning_rate = learning_rate, splitter = splitter, min_samples_leaf = min_samples_leaf, criterion = criterion , max_features = max_features)
gb_model.fit(X_train, y_train)

In [ ]:
# Предсказываем на тестовых данных
y_pred = gb_model.predict(X_test)

# Оцениваем качество модели
mse = mean_squared_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Mean Absolute Percentage Error:", mape)